In [1]:
!pip -qqq install langchain apify-client faiss-cpu sentence_transformers accelerate bitsandbytes
!pip -q install langchain unstructured sentence_transformers faiss-cpu huggingface_hub OpenAI
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate einops

from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import ApifyDatasetLoader
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 126.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

In [3]:
# ippbloader = ApifyDatasetLoader(
#     dataset_id="fksug1MdHtsrvFQvS",
#     dataset_mapping_function=lambda dataset_item: Document(
#         page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
#     ),
# )
# documents = ippbloader.load()

# text_splitter = CharacterTextSplitter(
#     separator = "\n",
#     chunk_size = 1000,
#     chunk_overlap  = 200
#     )

# docs = text_splitter.split_documents(documents)
# db = FAISS.from_documents(docs, embeddings)


!pip install pdfplumber
import pdfplumber

def pdf_loader(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages

        documents = []
        for page in pages:
            text = page.extract_text()
            documents.append(Document(page_content=text))

    return documents

documents = pdf_loader("/content/Postpartum_Depression.pdf")

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200
    )

docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.4 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [4]:
model = "Writer/camel-5b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
base_model = AutoModelForCausalLM.from_pretrained(
    model,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    # torch_dtype=torch.float16
)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# pipe = pipeline(
#     "text-generation",
#     model=base_model,
#     tokenizer=tokenizer,
#     max_length=3000,
#     temperature=0,
#     top_p=0.95,
#     repetition_penalty=1.2,
# )

pipe = pipeline(
    "text-generation", #task
    model=base_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=5000,
    do_sample=True,
    temperature=0.1,
    top_k=10,
    repetition_penalty=1.2,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


local_llm = HuggingFacePipeline(pipeline=pipe)
# local_llm

In [6]:
# qa = RetrievalQAWithSourcesChain.from_chain_type(llm=local_llm,
qa = RetrievalQA.from_chain_type(llm=local_llm,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(k=2),
                                 return_source_documents=True,
                                 verbose=True)

query = "what is postpartum depression"
result = qa(query)
result

Token indices sequence length is longer than the specified maximum sequence length for this model (855 > 512). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is postpartum depression',
 'result': ' Postpartum depression is characterized by persistent sadness, loss of interest in activities,\nexcessive crying, mood swings, feelings of guilt, anxiety, and difficulty caring for oneself.',
 'source_documents': [Document(page_content="7/20/23, 8:12 PM Postpartum Depression: Causes, Symptoms & Treatment\nIMPORTANT UPDATES + NOTICE OF VENDOR DATA EVENT\nPostpartum Depression\nPostpartum depression is a type of depression that happens after having a\nbaby. It affects up to 15% of people. People with postpartum depression\nexperience emotional highs and lows, frequent crying, fatigue, guilt, anxiety and\nmay have trouble caring for their baby. Postpartum depression can be treated\nwith medication and counseling.\nSymptoms and Causes Diagnosis and Tests Management and Treatment\nPrevention Outlook / Prognosis Living With Frequently Asked Questions\nOVERVIEW\nWhat is postpartum depression?\nHaving a baby is a life-changing experience. 